In [1]:
pwd

'E:\\Users\\Raphael\\Desktop\\Silent Night (Easy)'

In [2]:
home = _

In [16]:
import mido
from os.path import join as opj
import rtmidi
import xml.etree.ElementTree as ET

In [9]:
available_ports = rtmidi.MidiOut().get_ports()
available_ports

['Microsoft GS Wavetable Synth 0', 'V25 1', 'MIDIOUT2 (V25) 2']

In [10]:
port = available_ports[0]
port

'Microsoft GS Wavetable Synth 0'

In [13]:
mid = mido.MidiFile(opj(home,"Silent Night (Easy).mid"))
mid

<midi file 'E:\\Users\\Raphael\\Desktop\\Silent Night (Easy)\\Silent Night (Easy).mid' type 1, 2 tracks, 322 messages>

In [15]:
def play_song():
    with mido.open_output(port) as output:
        for msg in mid.play():
            data = msg.dict()
            if 'velocity' in data:
                vel = data['velocity']
                if vel > 0:
                    print('Note On')
                else:
                    print('Note Off')
            else:
                print('Not a note')
            output.send(msg)
play_song()

Not a note
Not a note
Not a note
Not a note
Not a note
Not a note
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note Off
Note On
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note Off
Note On
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note On
Note Off
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note Off
Note On
Note Off
Note Off
Note On
Note On
Note Off
Note On
Note Off
Note Off

In [26]:
with open(opj(home,'Silent Night (Easy).synthesia')) as xml:
    tree = ET.parse(xml)
    root = tree.getroot()
    for item in root.findall('./Songs'):
        for child in item:
            fingerhints = child.attrib['FingerHints']
right,left = fingerhints.split(' t1: ')
right,left

('9097909700800788098787688098787699097800760766',
 '53131531315313153131531315313153131543215215531315432152155313153131531315313153131531315313153215313153131153131531315')

In [27]:
def cvt_fingers(v):
    arr = []
    for cha in v:
        cha = int(cha)
        if cha > 5:
            cha -=5
        elif cha == 0:
            cha = 5
        arr += [cha]
    return arr

In [30]:
left,right = cvt_fingers(left),cvt_fingers(right)
left,right

([5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  4,
  3,
  2,
  1,
  5,
  2,
  1,
  5,
  5,
  3,
  1,
  3,
  1,
  5,
  4,
  3,
  2,
  1,
  5,
  2,
  1,
  5,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  2,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  1,
  5,
  3,
  1,
  3,
  1,
  5,
  3,
  1,
  3,
  1,
  5],
 [4,
  5,
  4,
  2,
  4,
  5,
  4,
  2,
  5,
  5,
  3,
  5,
  5,
  2,
  3,
  3,
  5,
  4,
  3,
  2,
  3,
  2,
  1,
  3,
  3,
  5,
  4,
  3,
  2,
  3,
  2,
  1,
  4,
  4,
  5,
  4,
  2,
  3,
  5,
  5,
  2,
  1,
  5,
  2,
  1,
  1])

In [48]:
lnotes = []
i = 0
for msg in mid.tracks[1]:
    if msg.type == 'note_on' and msg.dict()['velocity'] > 0:
        lnotes += [(msg,left[i])]
        i = i + 1
rnotes = []
i = 0
for msg in mid.tracks[0]:
    if msg.type == 'note_on' and msg.dict()['velocity'] > 0:
        rnotes += [(msg,right[i])]
        i = i + 1

In [49]:
lnotes,rnotes

([(<message note_on channel=0 note=48 velocity=80 time=0>, 5),
  (<message note_on channel=0 note=52 velocity=80 time=25>, 3),
  (<message note_on channel=0 note=55 velocity=80 time=0>, 1),
  (<message note_on channel=0 note=52 velocity=80 time=253>, 3),
  (<message note_on channel=0 note=55 velocity=80 time=0>, 1),
  (<message note_on channel=0 note=48 velocity=80 time=253>, 5),
  (<message note_on channel=0 note=52 velocity=80 time=25>, 3),
  (<message note_on channel=0 note=55 velocity=80 time=0>, 1),
  (<message note_on channel=0 note=52 velocity=80 time=253>, 3),
  (<message note_on channel=0 note=55 velocity=80 time=0>, 1),
  (<message note_on channel=0 note=48 velocity=80 time=253>, 5),
  (<message note_on channel=0 note=52 velocity=80 time=25>, 3),
  (<message note_on channel=0 note=55 velocity=80 time=0>, 1),
  (<message note_on channel=0 note=52 velocity=80 time=253>, 3),
  (<message note_on channel=0 note=55 velocity=80 time=0>, 1),
  (<message note_on channel=0 note=48 velo

In [57]:
def extract(v):
    arr = []
    for (msg,finger) in v:
        arr += [(msg.dict()['note'],msg.dict()['velocity'],finger)]
    return arr

In [58]:
lnotes_e,rnotes_e = extract(lnotes),extract(rnotes)

In [59]:
lnotes_e,rnotes_e

([(48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (43, 80, 5),
  (47, 80, 3),
  (50, 80, 1),
  (47, 80, 3),
  (50, 80, 1),
  (43, 80, 5),
  (47, 80, 3),
  (50, 80, 1),
  (47, 80, 3),
  (50, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (50, 80, 4),
  (52, 80, 3),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (50, 80, 4),
  (52, 80, 3),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80,